In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression
from torch_cluster import random_walk

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborSampler as RawNeighborSampler
from torch_geometric.nn import SAGEConv

In [6]:
EPS = 1e-15

dataset = 'Cora'
dataset = Planetoid("./Datasets", dataset, transform=T.NormalizeFeatures())
data = dataset[0]
print(data.x)
print(data.edge_index)
print(data.edge_weight)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
None


In [7]:
class NeighborSampler(RawNeighborSampler):
    def sample(self, batch):
        batch = torch.tensor(batch)
        row, col, _ = self.adj_t.coo()
        #print(self.adj_t)
        # For each node in `batch`, we sample a direct neighbor (as positive
        # example) and a random node (as negative example):
        pos_batch = random_walk(row, col, batch, walk_length=1,
                                coalesced=False)[:, 1]

        neg_batch = torch.randint(0, self.adj_t.size(1), (batch.numel(), ),
                                  dtype=torch.long)

        batch = torch.cat([batch, pos_batch, neg_batch], dim=0)
        return super().sample(batch)


train_loader = NeighborSampler(data.edge_index, sizes=[10, 10], batch_size=256,
                               shuffle=True, num_nodes=data.num_nodes)

In [8]:
class SAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(SAGEConv(in_channels, hidden_channels))

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

    def full_forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGE(data.num_node_features, hidden_channels=64, num_layers=2)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x, edge_index = data.x.to(device), data.edge_index.to(device)

In [9]:
def train():
    model.train()

    total_loss = 0
    for batch_size, n_id, adjs in train_loader:
        #print(batch_size,n_id,adjs)
        # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
        adjs = [adj.to(device) for adj in adjs]
        optimizer.zero_grad()

        out = model(x[n_id], adjs)
        out, pos_out, neg_out = out.split(out.size(0) // 3, dim=0)

        pos_loss = F.logsigmoid((out * pos_out).sum(-1)).mean()
        neg_loss = F.logsigmoid(-(out * neg_out).sum(-1)).mean()
        loss = -pos_loss - neg_loss
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * out.size(0)

    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    out = model.full_forward(x, edge_index).cpu()

    clf = LogisticRegression()
    clf.fit(out[data.train_mask], data.y[data.train_mask])

    val_acc = clf.score(out[data.val_mask], data.y[data.val_mask])
    test_acc = clf.score(out[data.test_mask], data.y[data.test_mask])

    return val_acc, test_acc


for epoch in range(1, 51):
    loss = train()
    val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Loss: 1.3873, Val: 0.3440, Test: 0.3460
Epoch: 002, Loss: 1.3197, Val: 0.5280, Test: 0.4860
Epoch: 003, Loss: 1.2018, Val: 0.6200, Test: 0.6120
Epoch: 004, Loss: 1.0985, Val: 0.7000, Test: 0.6890
Epoch: 005, Loss: 1.0495, Val: 0.7020, Test: 0.6920
Epoch: 006, Loss: 1.0284, Val: 0.6960, Test: 0.7170
Epoch: 007, Loss: 1.0251, Val: 0.7000, Test: 0.7110
Epoch: 008, Loss: 1.0022, Val: 0.6800, Test: 0.7030
Epoch: 009, Loss: 0.9900, Val: 0.7100, Test: 0.7160
Epoch: 010, Loss: 0.9836, Val: 0.7200, Test: 0.7330
Epoch: 011, Loss: 0.9760, Val: 0.7200, Test: 0.7380
Epoch: 012, Loss: 0.9748, Val: 0.7080, Test: 0.7350
Epoch: 013, Loss: 0.9669, Val: 0.7180, Test: 0.7370
Epoch: 014, Loss: 0.9716, Val: 0.7300, Test: 0.7290
Epoch: 015, Loss: 0.9533, Val: 0.7260, Test: 0.7270
Epoch: 016, Loss: 0.9444, Val: 0.7500, Test: 0.7400
Epoch: 017, Loss: 0.9596, Val: 0.7600, Test: 0.7440
Epoch: 018, Loss: 0.9323, Val: 0.7620, Test: 0.7530
Epoch: 019, Loss: 0.9448, Val: 0.7280, Test: 0.7450
Epoch: 020, 